In [ ]:
# ===== PHẦN 1.1: CÀI ĐẶT THƯ VIỆN =====
!pip install ultralytics -q
!pip install deep-sort-realtime -q
!pip install transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 72.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ===== PHẦN 1.2: TẢI WEIGHTS =====
import os

WEIGHTS_DIR = 'weights'
WEIGHTS_FILE = os.path.join(WEIGHTS_DIR, 'best.pt')  
FILE_ID = '1TgD8SUa-hvtHVH-ligQJqrxhzYwzr-Lh'     

#
os.makedirs(WEIGHTS_DIR, exist_ok=True)

if not os.path.exists(WEIGHTS_FILE):
    print(f"🔄Đang tải weights từ Google Drive vào {WEIGHTS_FILE}...")
    try:
        import gdown
    except ImportError:
        !pip install gdown -q
        import gdown
    
    # Tải file
    url = f'https://drive.google.com/uc?id={FILE_ID}'
    gdown.download(url, WEIGHTS_FILE, quiet=False)
    print("\n Tải xong weights!")
else:
    print(" Đã tìm thấy file weights. Sẵn sàng chạy!")

In [ ]:
# ===== PHẦN 2: IMPORT VÀ SETUP =====
import cv2
import torch
import numpy as np
import torch.nn as nn
from PIL import Image
from ultralytics import YOLO
from transformers import CLIPProcessor, CLIPModel
from deep_sort_realtime.deepsort_tracker import DeepSort
import gc


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Using device: cuda


In [ ]:
import ultralytics.nn.modules as nn_mod
import ultralytics.nn.tasks as y_tasks


class CoordAtt(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        mip = max(8, channels // reduction)

        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))

        self.conv1 = nn.Conv2d(channels, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = nn.SiLU()

        self.conv_h = nn.Conv2d(mip, channels, kernel_size=1, stride=1, padding=0)
        self.conv_w = nn.Conv2d(mip, channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        identity = x
        n, c, h, w = x.size()

        x_h = self.pool_h(x)
        x_w = self.pool_w(x).permute(0, 1, 3, 2)

        y = torch.cat([x_h, x_w], dim=2)
        y = self.act(self.bn1(self.conv1(y)))

        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_w = x_w.permute(0, 1, 3, 2)

        a_h = torch.sigmoid(self.conv_h(x_h))
        a_w = torch.sigmoid(self.conv_w(x_w))

        return identity * a_h * a_w


nn_mod.CoordAtt = CoordAtt
y_tasks.CoordAtt = CoordAtt
globals()['CoordAtt'] = CoordAtt

print(" Custom CoordAtt module registered successfully!")

In [ ]:
# ===== PHẦN 3: LOAD MODELS =====
print(" Loading YOLO model...")
try:
    yolo = YOLO("/content/drive/MyDrive/cpv/39_epoch_best8-7.pt")  # Đường dẫn model của bạn trong Drive
    print(" Custom YOLO model loaded")
except:
    yolo = YOLO("yolov8n.pt")
    print(" Using default YOLOv8n")

print(" Loading CLIP model...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()
print(" CLIP model loaded")

In [ ]:
# ===== PHẦN 4: ENCODE TEXT & IMAGE REFERENCE =====
text_query = "a car light orange and yellow"
print(f" Text query: {text_query}")

text_inputs = clip_processor(text=[text_query], return_tensors="pt", padding=True).to(device)
with torch.no_grad():
    text_emb = clip_model.get_text_features(**text_inputs)
    text_emb = text_emb / text_emb.norm(p=2, dim=1, keepdim=True)

🔍 Text query: a car light orange and yellow


In [ ]:
# Encode ảnh tham chiếu
reference_image_path = "/content/drive/MyDrive/cpv/x_force.jpg"  # Đường dẫn ảnh xe trong Drive
ref_image = Image.open(reference_image_path).convert("RGB").resize((224, 224))
ref_inputs = clip_processor(images=[ref_image], return_tensors="pt", padding=True).to(device)
with torch.no_grad():
    ref_emb = clip_model.get_image_features(**ref_inputs)
    ref_emb = ref_emb / ref_emb.norm(p=2, dim=1, keepdim=True)
print(" Reference embeddings ready")

✅ Reference embeddings ready


In [ ]:

# ===== PHẦN 5: KHỞI TẠO DEEPSORT TRACKER =====
tracker = DeepSort(
    max_age=30,           # Số frame tối đa giữ track khi mất
    n_init=3,             # Số frame khởi tạo track
    nms_max_overlap=0.7,  # NMS threshold
    max_cosine_distance=0.3,
    nn_budget=100
)
print(" DeepSort tracker initialized")

# ===== PHẦN 6: CẤU HÌNH VIDEO =====
video_path = "./inputs/vd_1.mp4"  # Video input trong Drive
output_path = "./outputs/output_tracking.mp4"  # Video output

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Resize để giảm tải GPU/RAM
RESIZE_WIDTH = 640
RESIZE_HEIGHT = int(height * (RESIZE_WIDTH / width))

out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (RESIZE_WIDTH, RESIZE_HEIGHT)
)

print(f" Video: {width}x{height} @ {fps}fps ({total_frames} frames)")
print(f" Resize to: {RESIZE_WIDTH}x{RESIZE_HEIGHT}")

# ===== PHẦN 7: CẤU HÌNH MATCHING =====
THRESHOLD_TEXT = 0.22      # Ngưỡng text similarity
THRESHOLD_IMAGE = 0.44     # Ngưỡng image similarity
MATCH_INTERVAL = 5        # Số frame check matching một lần
MIN_TRACK_FRAMES = 2       # Số frame tối thiểu trước khi match

# Lưu trữ tracks và matching results
tracked_vehicles = {}      # {track_id: {'embeddings': [], 'matched': False, 'best_sim': 0}}
target_track_ids = set()   # Các track_id đã match với xe cần tìm

# ===== PHẦN 8: HÀM EXTRACT CLIP FEATURES =====
@torch.no_grad()
def extract_clip_features(crops):
    """Extract CLIP features cho batch crops"""
    if not crops:
        return []

    images = [Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)).resize((224, 224))
              for crop in crops]
    inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(device)
    features = clip_model.get_image_features(**inputs)
    features = features / features.norm(p=2, dim=1, keepdim=True)
    return features

# ===== PHẦN 9: HÀM MATCHING =====
def check_track_matching(track_id, track_data):
    """Kiểm tra track có match với xe cần tìm không"""
    if track_data['matched'] or len(track_data['embeddings']) < MIN_TRACK_FRAMES:
        return False

    # Lấy embedding trung bình từ các frame
    avg_emb = torch.mean(torch.stack(track_data['embeddings']), dim=0, keepdim=True)

    # Tính similarity với text và reference image
    text_sim = (avg_emb @ text_emb.T).item()
    ref_sim = (avg_emb @ ref_emb.T).item()

    # Update best similarity
    combined_sim = (text_sim + ref_sim) / 2
    track_data['best_sim'] = max(track_data['best_sim'], combined_sim)

    # Check ngưỡng
    if text_sim >= THRESHOLD_TEXT and ref_sim >= THRESHOLD_IMAGE:
        track_data['matched'] = True
        return True

    return False

# ===== PHẦN 10: VÒNG LẶP CHÍNH =====
frame_count = 0
process_every_n = 1  # Xử lý mọi frame (có thể tăng lên 2-3 để tăng tốc)

print("\n🎬 Bắt đầu xử lý video...")
print("=" * 60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    
    if frame_count % process_every_n != 0:
        continue

    
    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    # ===== BƯỚC 1: YOLO DETECTION =====
    results = yolo(frame, conf=0.35, verbose=False)[0]

    # ===== BƯỚC 2: CHUẨN BỊ DETECTIONS CHO DEEPSORT =====
    detections = []
    crops = []
    bboxes_info = []

    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()
        cls = int(box.cls[0].cpu().numpy())

        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Validate bbox
        if x1 >= x2 or y1 >= y2 or x1 < 0 or y1 < 0:
            continue

        # DeepSORT format: ([left, top, width, height], confidence, class)
        detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

        # Crop image cho CLIP
        crop = frame[y1:y2, x1:x2]
        if crop.size > 0:
            crops.append(crop)
            bboxes_info.append((x1, y1, x2, y2))

    # ===== BƯỚC 3: UPDATE TRACKER =====
    tracks = tracker.update_tracks(detections, frame=frame)

    # ===== BƯỚC 4: EXTRACT CLIP FEATURES CHO CÁC CROPS =====
    if crops and frame_count % MATCH_INTERVAL == 0:
        clip_features = extract_clip_features(crops)

        # Map features to tracks
        feature_idx = 0
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id

            # Khởi tạo track data nếu chưa có
            if track_id not in tracked_vehicles:
                tracked_vehicles[track_id] = {
                    'embeddings': [],
                    'matched': False,
                    'best_sim': 0.0
                }

            # Thêm embedding mới (giới hạn 20 embeddings gần nhất)
            if feature_idx < len(clip_features):
                tracked_vehicles[track_id]['embeddings'].append(clip_features[feature_idx])
                if len(tracked_vehicles[track_id]['embeddings']) > 20:
                    tracked_vehicles[track_id]['embeddings'].pop(0)
                feature_idx += 1

            # Kiểm tra matching
            if check_track_matching(track_id, tracked_vehicles[track_id]):
                target_track_ids.add(track_id)
                print(f"🎯 Frame {frame_count}: Found target vehicle - Track ID {track_id} "
                      f"(Similarity: {tracked_vehicles[track_id]['best_sim']:.3f})")

    # ===== BƯỚC 5: VẼ BOUNDING BOXES =====
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        # Chọn màu và style dựa trên matching
        if track_id in target_track_ids:
            #  XE CẦN TÌM - Màu đỏ, bbox dày
            color = (0, 0, 255)      # Đỏ
            thickness = 4
            label = f"TARGET #{track_id}"
            label_bg = (0, 0, 255)
            label_text_color = (255, 255, 255)
        else:
            # Xe thông thường - Màu xanh lá
            color = (0, 255, 0)
            thickness = 2
            label = f"ID {track_id}"
            label_bg = (0, 255, 0)
            label_text_color = (0, 0, 0)

        # Vẽ bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

        # Vẽ label với background
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - h - 10), (x1 + w, y1), label_bg, -1)
        cv2.putText(frame, label, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, label_text_color, 2)

    # ===== BƯỚC 6: VẼ INFO PANEL =====
    info_text = [
        f"Frame: {frame_count}/{total_frames}",
        f"Tracked: {len([t for t in tracks if t.is_confirmed()])}",
        f"Targets: {len(target_track_ids)}"
    ]

    y_offset = 30
    for text in info_text:
        cv2.putText(frame, text, (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        y_offset += 30

    # ===== BƯỚC 7: WRITE FRAME =====
    out.write(frame)

    # Progress update
    if frame_count % 50 == 0:
        progress = (frame_count / total_frames) * 100
        print(f" Progress: {progress:.1f}% | Targets found: {len(target_track_ids)}")

    # Giải phóng RAM định kỳ
    if frame_count % 100 == 0:
        gc.collect()
        torch.cuda.empty_cache()

# ===== PHẦN 11: CLEANUP =====
cap.release()
out.release()
gc.collect()
torch.cuda.empty_cache()

print("\n" + "=" * 60)
print(" HOÀN THÀNH!")
print(f" Video output: {output_path}")
print(f" Tổng số xe cần tìm phát hiện: {len(target_track_ids)}")
if target_track_ids:
    print(f" Track IDs: {sorted(target_track_ids)}")
print("=" * 60)

✅ DeepSort tracker initialized
📹 Video: 576x1024 @ 29fps (1870 frames)
📏 Resize to: 640x1137

🎬 Bắt đầu xử lý video...
🎯 Frame 50: Found target vehicle - Track ID 2 (Similarity: 0.341)
⏳ Progress: 2.7% | Targets found: 1
🎯 Frame 75: Found target vehicle - Track ID 1 (Similarity: 0.330)
🎯 Frame 90: Found target vehicle - Track ID 20 (Similarity: 0.342)
⏳ Progress: 5.3% | Targets found: 3
⏳ Progress: 8.0% | Targets found: 3
⏳ Progress: 10.7% | Targets found: 3
🎯 Frame 205: Found target vehicle - Track ID 29 (Similarity: 0.338)
🎯 Frame 250: Found target vehicle - Track ID 26 (Similarity: 0.334)
⏳ Progress: 13.4% | Targets found: 5
⏳ Progress: 16.0% | Targets found: 5
⏳ Progress: 18.7% | Targets found: 5
🎯 Frame 380: Found target vehicle - Track ID 40 (Similarity: 0.335)
⏳ Progress: 21.4% | Targets found: 6
🎯 Frame 420: Found target vehicle - Track ID 70 (Similarity: 0.349)
🎯 Frame 450: Found target vehicle - Track ID 63 (Similarity: 0.337)
⏳ Progress: 24.1% | Targets found: 8
🎯 Frame 470: